# Analyza liekov podla skupiny ATC7 
## Otazky:
* Ktore lieky s danou ucinnou latkou sa predavaju viac ako ostatne? Zavisi to od ceny, ktoru pacient zaplati alebo od krajiny vyrobcu?

 1409 roznych ucinnych latok

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px
%run tables.py

In [3]:
table = import_table('detailed', 2019)
table2019=table.loc[:,['ATC7_KOD','ATC7_popis','LIEK_KOD','LIEK_popis', 'STAT_popis', 'VYROBCA_popis', 'Počet balení', 'Úhrada_PAC']]
display(table2019)

,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC
0,A01AA03,Olaflur,C40630,ELMEX FLUID sol dnt 1x50 ml,Nemecko,GABA GmbH,3.000,0.04
1,A01AA03,Olaflur,C40630,ELMEX FLUID sol dnt 1x50 ml,Nemecko,GABA GmbH,37.000,135.04
2,A01AA30,"Profylaxia zubného kazu, kombinácie",C40631,ELMEX gelee gel dnt 1x25 g,Nemecko,CP GABA GmbH,36.000,0.00
3,A01AA30,"Profylaxia zubného kazu, kombinácie",C40631,ELMEX gelee gel dnt 1x25 g,Nemecko,CP GABA GmbH,1394.000,0.00
4,A01AA30,"Profylaxia zubného kazu, kombinácie",C40631,ELMEX gelee gel dnt 1x25 g,Nemecko,CP GABA GmbH,99.000,678.87
...,...,...,...,...,...,...,...,...
32304,(prazdne),(prazdne),!,KÓD NIE JE V ČÍSELNÍKU,(prazdne),(prazdne),12.000,0.00
32305,(prazdne),(prazdne),!,KÓD NIE JE V ČÍSELNÍKU,(prazdne),(prazdne),755.350,0.00
32306,(prazdne),(prazdne),!,KÓD NIE JE V ČÍSELNÍKU,(prazdne),(prazdne),1321.300,6248.05
32307,(prazdne),(prazdne),!,KÓD NIE JE V ČÍSELNÍKU,(prazdne),(prazdne),403.000,0.00


In [4]:
table = import_table('detailed', 2019)
table2019=table.loc[:,['ATC7_KOD','ATC7_popis','LIEK_KOD','LIEK_popis', 'STAT_popis', 'VYROBCA_popis', 'Počet balení','Úhrada_PAC']]
table2019 = table2019.query('ATC7_KOD != "(prazdne)"').query( 'ATC7_popis != "neznámy"')
table2019sum = table2019.groupby(['ATC7_KOD','ATC7_popis','LIEK_KOD','LIEK_popis', 'STAT_popis','VYROBCA_popis']).sum()
table2019sum['Cena úhrada_PAC/bal'] = table2019sum['Úhrada_PAC'] / table2019sum['Počet balení']
atc7 = table2019.groupby(['ATC7_KOD', 'ATC7_popis']).sum().sort_values(by='Počet balení', ascending=False).reset_index()
display(table2019sum)

nazvy = []
for n in atc7['ATC7_KOD']:
    if n not in nazvy:
        nazvy.append(n)

top20tab= {}
for i in range(20):
    top20tab[nazvy[i]] = table2019sum.query(f'ATC7_KOD == "{nazvy[i]}"').sort_values(by='Počet balení', ascending= False).reset_index()
    display(top20tab[nazvy[i]].head())

Počet balení  \
ATC7_KOD ATC7_popis                          LIEK_KOD LIEK_popis                       STAT_popis VYROBCA_popis                                                      
A01AA03  Olaflur                             C40630   ELMEX FLUID sol dnt 1x50 ml      Nemecko    GABA GmbH                                                   40.0   
A01AA30  Profylaxia zubného kazu, kombinácie C40631   ELMEX gelee gel dnt 1x25 g       Nemecko    CP GABA GmbH                                             43832.5   
                                             C85867   ELMEX gelee gel dnt 1x215 g      Nemecko    CP GABA GmbH                                                29.0   
A01AB03  Chlorhexidín                        C14106   CORSODYL sol ora 1x600 ml        Česko      GlaxoSmithKline Consumer Healthcare Czech Repub...          72.0   
                                             C34296   PROTEFIX ANTISEPT sol der 200 ml Nemecko    Queisser Pharma GmbH & Co.                                   6.0   
...                                                                                                                                                            ...   
V12      Homeopatiká                         C82842   GINKGO BILOBA plv 15 g ch2-ch200 Francúzsko Laboratoires BOIRON                                        309.0   
                                             C96701   ARNICA MONTANA ung 20 g ch2-ch30 Francúzsko Laboratoires BOIRON                                         15.0   
                                             C97944   QUIETUDE tbl 60                  Francúzsko Laboratoires BOIRON                                        156.0   
                                             C99176   BELLADONNA gra 4 g ch3-ch30      Francúzsko Laboratoires BOIRON                                        183.0   
                                             C99177   ARNICA MONTANA gra 4 g ch2-ch30  Francúzsko Laboratoires BOIRON                                        700.0   

                                                                                                                                                      Úhrada_PAC  \
ATC7_KOD ATC7_popis                          LIEK_KOD LIEK_popis                       STAT_popis VYROBCA_popis                                                    
A01AA03  Olaflur                             C40630   ELMEX FLUID sol dnt 1x50 ml      Nemecko    GABA GmbH                                               135.08   
A01AA30  Profylaxia zubného kazu, kombinácie C40631   ELMEX gelee gel dnt 1x25 g       Nemecko    CP GABA GmbH                                         313077.14   
                                             C85867   ELMEX gelee gel dnt 1x215 g      Nemecko    CP GABA GmbH                                            171.85   
A01AB03  Chlorhexidín                        C14106   CORSODYL sol ora 1x600 ml        Česko      GlaxoSmithKline Consumer Healthcare Czech Repub...      254.33   
                                             C34296   PROTEFIX ANTISEPT sol der 200 ml Nemecko    Queisser Pharma GmbH & Co.                               27.82   
...                                                                                                                                                          ...   
V12      Homeopatiká                         C82842   GINKGO BILOBA plv 15 g ch2-ch200 Francúzsko Laboratoires BOIRON                                     882.18   
                                             C96701   ARNICA MONTANA ung 20 g ch2-ch30 Francúzsko Laboratoires BOIRON                                      52.29   
                                             C97944   QUIETUDE tbl 60                  Francúzsko Laboratoires BOIRON                                    1030.21   
                                             C99176   BELLADONNA gra 4 g ch3-ch30      Francúzsko Laboratoires BOIRON                                     631.73   
                       

,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,V09FX01,Technécium - (,C91102,"Drytec gen rnk 2,5 GBq",Holandsko,GE Healthcare B.V.,6688913.50,0.0,0.0
1,V09FX01,Technécium - (,C15457,"Drytec gen rnk 10,0 GBq",Holandsko,GE Healthcare B.V.,1469870.00,0.0,0.0
2,V09FX01,Technécium - (,C15308,"Ultra-TechneKow FM (99mTC-generátor) inj 10,63...",Holandsko,Curium Netherlands B.V.,991979.00,0.0,0.0
3,V09FX01,Technécium - (,C15456,"Drytec gen rnk 9,0 GBq",Holandsko,GE Healthcare B.V.,467689.01,0.0,0.0
4,V09FX01,Technécium - (,C15307,"Ultra-TechneKow FM (99mTC-generátor) inj 8,50 ...",Holandsko,Curium Netherlands B.V.,14.00,0.0,0.0


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,B05BB01,Elektrolyty,C37743,"0,9% SODIUM CHL.IN W.F.I.FRESENIUS sol inf 1x1...",Česko,Fresenius Kabi s.r.o.,1115466.27,1758.64,0.001577
1,B05BB01,Elektrolyty,C33593,"0,9% SODIUM ChL.IN W.F.I.FRESENIUS sol inf 1x2...",Česko,Fresenius Kabi s.r.o.,999336.36,4893.52,0.004897
2,B05BB01,Elektrolyty,C33590,"0,9% SODIUM ChL.IN W.F.I.FRESENIUS sol inf 1x5...",Česko,Fresenius Kabi s.r.o.,789546.85,1375.02,0.001742
3,B05BB01,Elektrolyty,C97748,"SODIUM CHLORIDE 0,9%W/V I.V. INF inf 1x100 ml ...",Taliansko,Bieffe Medital S.p.A.,418659.15,5287.02,0.012628
4,B05BB01,Elektrolyty,C97749,"SODIUM CHLORIDE 0,9%W/V I.V. INF inf 1x250 ml ...",Taliansko,Bieffe Medital S.p.A.,381477.64,18777.30,0.049223


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,N02BE01,Paracetamol,C86052,PARALEN 500 tbl 24x500 mg (blis.Al/PVC),Slovensko,sanofi-aventis Slovakia s.r.o.,2300126.084,3826254.68,1.663498
1,N02BE01,Paracetamol,C86051,PARALEN 500 tbl 12x500 mg (blis.Al/PVC),Slovensko,sanofi-aventis Slovakia s.r.o.,269140.730,279950.53,1.040164
2,N02BE01,Paracetamol,C27613,PARALEN 100 sup 5x100 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,139481.200,117364.89,0.841439
3,N02BE01,Paracetamol,C57476,Panadol Paracetamolum 500 mg tbl flm 24x500 mg...,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,113876.250,194443.83,1.707501
4,N02BE01,Paracetamol,C26627,PARALEN 125 tbl 20x125 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,111906.328,81286.36,0.726379


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,M01AE01,Ibuprofén,C35458,IBALGIN 400 tbl flm 24x400 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,596621.970,1885611.99,3.160480
1,M01AE01,Ibuprofén,C7880A,Ibalgin 400 tbl flm 48x400 mg (blis. PVC/Al),Slovensko,sanofi-aventis Slovakia s.r.o.,293384.731,1432962.47,4.884244
2,M01AE01,Ibuprofén,C97299,BRUFEN 400 tbl flm 30x400 mg,Francúzsko,Mylan Laboratories SAS,232798.061,868313.80,3.729901
3,M01AE01,Ibuprofén,C49851,MIG-400 tbl flm 30x400 mg,Nemecko,Berlin-Chemie AG,205849.323,706016.17,3.429772
4,M01AE01,Ibuprofén,C45451,NUROFEN pre deti Jahoda sus por 1x100 ml (PET ...,Česko,"Reckitt Benckiser (Czech Republic), spol. s r.o.",160812.710,747388.09,4.647569


...

,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C65951,TheraFlu prechladnutie a chrípka plo por 1x14 ...,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,422718.171,3954421.86,9.354748
1,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C30229,PARALEN PLUS tbl flm 24,Slovensko,sanofi-aventis Slovakia s.r.o.,100459.030,589486.22,5.867927
2,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C33813,THERAFLU PRECHLADNUTIE A CHRÍPKA plo por 1x10 ...,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,81452.991,671984.16,8.249963
3,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C36516,PARALEN HOT DRINK plu por 1x12 vreciek,Slovensko,sanofi-aventis Slovakia s.r.o.,65515.562,481851.44,7.354763
4,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C0381A,PARALEN GRIP chrípka a bolesť tbl flm 1x24 (bl...,Slovensko,sanofi-aventis Slovakia s.r.o.,61588.890,362517.93,5.886093


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,M02AA15,Diklofenak,C11849,"Voltaren Forte 2,32 % gél gel der 1x100 g (tub...",Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,189743.17,2256703.40,11.893463
1,M02AA15,Diklofenak,C11851,"Voltaren Forte 2,32 % gél gel der 1x150 g (tub...",Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,136412.90,2083185.45,15.271176
2,M02AA15,Diklofenak,C18515,FLECTOR EP TISSUGEL emp tdm 5,Slovensko,IBSA Slovakia s.r.o.,136175.20,1201954.19,8.826528
3,M02AA15,Diklofenak,C18514,FLECTOR EP gel 1x100 g,Slovensko,IBSA Slovakia s.r.o.,112500.25,884399.45,7.861311
4,M02AA15,Diklofenak,C84453,DICLOBENE gel 1x100 g,Holandsko,TEV-NL-4?,94815.70,515849.66,5.440551


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,A11GA01,Kyselina askorbová (vitamín C),C78278,CELASKON 250 tbl 100x250 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,396438.278,1796114.28,4.530628
1,A11GA01,Kyselina askorbová (vitamín C),C78277,CELASKON 250 tbl 30x250 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,250880.500,389292.20,1.551704
2,A11GA01,Kyselina askorbová (vitamín C),C42420,CELASKON 500 mg červený pomaranč tbl eff 30x50...,Slovensko,sanofi-aventis Slovakia s.r.o.,106336.777,602810.11,5.668877
3,A11GA01,Kyselina askorbová (vitamín C),C35913,CELASKON LONG EFFECT cps pld 60x500 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,79906.524,654347.40,8.188911
4,A11GA01,Kyselina askorbová (vitamín C),C24836,ACIDUM ASCORBICUM BIOTIKA sol inj 5x5 ml/500 mg,Česko,BB Pharma a.s.,79579.400,42800.62,0.537835


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,A02BC02,Pantoprazol,C43345,Nolpaza 20 mg tbl ent 30x20 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",184810.510,255770.78,1.383962
1,A02BC02,Pantoprazol,C43349,Nolpaza 20 mg tbl ent 100x20 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",144399.293,632243.24,4.378437
2,A02BC02,Pantoprazol,C43362,Nolpaza 40 mg tbl ent 100x40 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",117848.280,648739.25,5.504868
3,A02BC02,Pantoprazol,C43358,Nolpaza 40 mg tbl ent 30x40 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",107996.080,297846.23,2.757936
4,A02BC02,Pantoprazol,C43791,Pantoprazol Sandoz 20 mg gastrorezistentné tab...,Slovinsko,Sandoz Pharmaceuticals d.d.,73014.340,105977.50,1.451461


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,R01AA09,Tramazolín,C26797,MUCONASAL PLUS sol neb 1x10 ml,Slovensko,sanofi-aventis Slovakia s.r.o.,1135312.849,5548815.77,4.887477


## Ukazky neupravenych grafov

In [5]:
for i in range(20):
    fig = px.bar(top20tab[nazvy[i]], x="Počet balení", y="LIEK_KOD", color="STAT_popis", title=nazvy[i], orientation='h')
    #fig.show()

# Grafy jednotlivych ucinnych latok:

# Technecium

In [6]:
technecium = top20tab["V09FX01"]

for i,popis in enumerate(technecium['LIEK_popis']):
    if 'Drytec gen' in popis:
        technecium.iloc[i,3] = 'Drytec gen rnk'
    elif "Ultra-TechneKow" in popis:
        technecium.iloc[i,3] = 'Ultra-TechneKow FM (99mTC-generátor)'

technecium = technecium.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(technecium)
print("Typ vydaju:")
item_counts = table.query('ATC7_KOD == "V09FX01"')["TYP_VYDAJ_POP"].value_counts(normalize=True)
print(item_counts)


,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,Drytec gen rnk,Holandsko,GE Healthcare B.V.,8626479.51,0.0,0.0
1,Ultra-TechneKow FM (99mTC-generátor),Holandsko,Curium Netherlands B.V.,992000.00,0.0,0.0


Typ vydaju:
výdaj z nemocničnej lekárne pre vlastnú nemocnicu    0.692308
úhrada z verejnej poisťovne                          0.307692
Name: TYP_VYDAJ_POP, dtype: float64


In [7]:
fig = px.bar(technecium, x='LIEK_popis', y='Počet balení', color='VYROBCA_popis', title='Technecium')
fig.show()

 Na vizualizaciu ucinnej latky technecium nam bolo najvyhodnejsie pouzit barplot. 
 Nakolko sa jedna o diagnosticke radiofarmaka, podla vypisu Series vidime ze tieto lieky boli poskytovane nemocnicou alebo ich
 pacientom hradila zdravotna poistovna. V oboch pripadoch pacient neplatil nic, a tak udaje o cene nemame ako zakomponovat. 

# Elektrolyty: ZLE treba prerobit

In [14]:
elektrolyty = top20tab["B05BB01"].head(30)
'''
for i,popis in enumerate(elektrolyty['LIEK_popis']):
    if '0,9% SODIUM CHL.IN' in popis or '0,9% SODIUM ChL.IN' in popis:
        elektrolyty.iloc[i,3] = '0,9% SODIUM CHL.IN W.F.I.FRESENIUS sol inf'
    elif "SODIUM CHLORIDE 0,9%W/V I.V. INF" in popis:
        elektrolyty.iloc[i,3] = 'SODIUM CHLORIDE 0,9%W/V I.V. INF'
    elif "0,9% W/V SODIUM CHLORIDE I.V." in popis:
        elektrolyty.iloc[i,3] = '0,9% W/V SODIUM CHLORIDE I.V.'
    elif "PLASMALYTE sol inf" in popis:
        elektrolyty.iloc[i,3] = 'PLASMALYTE sol inf'
    elif "INFUSIO NATRII CHLORATI ISOTONICA IMUNA sol inf" in popis:
        elektrolyty.iloc[i,3] = 'INFUSIO NATRII CHLORATI ISOTONICA IMUNA sol inf'
    elif "RINGER'S INJECTION 'FRESENIUS' sol inf" in popis:
        elektrolyty.iloc[i,3] = "RINGER'S INJECTION 'FRESENIUS' sol inf"
    elif "HARTMANN'S SOLUTION BP FRESENIUS" in popis:
        elektrolyty.iloc[i,3] = "HARTMANN'S SOLUTION BP FRESENIUS sol inf"
    elif 'Chlorid sodný 0,9 % IMUNA ENVIBAG sol inf' in popis:
        elektrolyty.iloc[i,3] = 'Chlorid sodný 0,9 % IMUNA ENVIBAG sol inf'
    elif 'Isolyte sol inf' in popis:
        elektrolyty.iloc[i,3] = 'Isolyte sol inf'
    elif 'RINGEROV ROZTOK BAXTER-VIAFLO sol inf' in popis:
        elektrolyty.iloc[i,3] = 'RINGEROV ROZTOK BAXTER-VIAFLO sol inf'
    elif "SOLUTION BP" in popis:
        elektrolyty.iloc[i,3] = "HARTMANN'S SOLUTION BP Fresenius"
    elif "RINGER'S INJECTION 'FRESENIUS' sol inf" in popis:
        elektrolyty.iloc[i,3] = "RINGER'S INJECTION 'FRESENIUS' sol inf"
    elif 'INJECTION FRESENIUS"""""' in popis:
        elektrolyty.iloc[i,3] = "RINGER'S INJECTION 'FRESENIUS' sol inf"
'''       
elektrolyty = elektrolyty.groupby(['LIEK_KOD','LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index().query('Počet balení > 80000')
display(elektrolyty.sort_values(by='LIEK_popis'))
fig = px.scatter(elektrolyty, x='LIEK_KOD', y='Počet balení', size='Cena úhrada_PAC/bal', color='STAT_popis', title='Elektrolyty')
fig.show()


SyntaxError: invalid syntax (<unknown>, line 1)

# Paracetamol

In [9]:
paracetamol = top20tab["N02BE01"].sort_values(by='Počet balení', ascending=False).head(30)

for i,popis in enumerate(paracetamol['LIEK_popis']):
    if 'MEDIPYRIN 500 tbl' in popis:
        paracetamol.iloc[i,3] = 'MEDIPYRIN tbl'
    elif 'PANADOL BABY (+odmerka)' in popis:
        paracetamol.iloc[i,3] = 'PANADOL BABY (+odmerka)'
    elif 'PANADOL BABY (+striekačka)' in popis:
        paracetamol.iloc[i,3] = 'PANADOL BABY (+striekačka)'
    elif 'PANADOL BABY' in popis:
        paracetamol.iloc[i,3] = 'PANADOL BABY'
    elif 'PANADOL JUNIOR' in popis:
        paracetamol.iloc[i,3] = 'PANADOL JUNIOR'
    elif 'PANADOL PRE DETI' in popis:
        paracetamol.iloc[i,3] = 'PANADOL PRE DETI'
    elif 'PANADOL tbl' in popis:
        paracetamol.iloc[i,3] = 'PANADOL tbl'
    elif 'PARALEN' in popis and 'sus' in popis:
        paracetamol.iloc[i,3] = 'PARALEN sus'
    elif 'PARALEN' in popis and 'tbl' in popis:
        paracetamol.iloc[i,3] = 'PARALEN tbl'
    elif 'PARALEN' in popis and 'sup' in popis:
        paracetamol.iloc[i,3] = 'PARALEN sup'
    elif 'PARALEN' in popis:
        paracetamol.iloc[i,3] = 'PARALEN nápoj'
    elif 'Panadol Novum' in popis:
        paracetamol.iloc[i,3] = 'Panadol Novum tbl'
    elif 'Panadol Paracetamolum' in popis:
        paracetamol.iloc[i,3] = 'Panadol Paracetamolum tbl'
    elif 'Paracetamol B. Braun' in popis:
        paracetamol.iloc[i,3] = 'Paracetamol B. Braun'
    elif 'Paracetamol Kabi' in popis:
        paracetamol.iloc[i,3] = 'Paracetamol Kabi'
    elif 'Paracetamol Kappler' in popis:
        paracetamol.iloc[i,3] = 'Paracetamol Kappler'
    elif 'Paracetamol Vitabalans' in popis:
        paracetamol.iloc[i,3] = 'Paracetamol Vitabalans'

paracetamol = paracetamol.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()

print("Typ vydaju:")
item_counts = table.query('ATC7_KOD == "N02BE01"')["TYP_VYDAJ_POP"].value_counts(normalize=True)
print(item_counts)

display(paracetamol.sort_values(by='LIEK_popis'))


Typ vydaju:
predané z verejnej lekárne bez receptu občanom (voľnopredajné lieky)    0.304054
výdaj z verejnej lekárne  na recept bez úhrady z verejného poistenia    0.209459
výdaj z nemocničnej lekárne pre vlastnú nemocnicu                       0.162162
výdaj z verejnej lekárne na faktúru do neštátnych ambulancií            0.162162
výdaj z verejnej lekárne na žiadanku nemocniciam                        0.141892
výdaj z nemocničnej lekárne pre zmluvnú nemocnicu                       0.013514
úhrada z verejnej poisťovne                                             0.006757
Name: TYP_VYDAJ_POP, dtype: float64


,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,MEDIPYRIN tbl,Česko,Glenmark Pharmaceuticals s.r.o.,21920.890,36758.59,4.625888
1,PANADOL BABY,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,50394.000,100522.52,1.994732
2,PANADOL BABY (+odmerka),Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,839.000,2862.69,3.412026
3,PANADOL BABY (+striekačka),Francúzsko,SmithKline Beecham Liquides Industrie,285.000,1012.17,3.551474
4,PANADOL JUNIOR,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,30460.500,77105.53,2.531328
5,PANADOL PRE DETI,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,37895.000,140531.82,7.440675
6,PANADOL tbl,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,553.000,942.77,2.667794
7,PARALEN nápoj,Slovensko,sanofi-aventis Slovakia s.r.o.,482.170,1673.49,3.470747
8,PARALEN sup,Slovensko,sanofi-aventis Slovakia s.r.o.,161406.210,153352.77,2.482846
9,PARALEN sus,Slovensko,sanofi-aventis Slovakia s.r.o.,74950.000,270530.14,3.609475


In [35]:
fig = px.scatter(paracetamol, x='LIEK_popis', y='Počet balení', size='Cena úhrada_PAC/bal', color='STAT_popis', title='Paracetamol')
fig.update_xaxes(tickangle=45)
fig.show()

Z grafu vidime, ze vsetky lieky prekonava Paralen v tabletach za priblizne priemernu cenu. Takisto si mozeme vsimnut ze je vyrabany na Slovensku. Cesko ma siroku ponuku Panadolov avsak tieto tak popularne nie su.

In [28]:
paralen_spolu = paracetamol.sort_values(by='LIEK_popis').iloc[7:11,:].groupby('STAT_popis').sum()
paralen = paralen_spolu.iloc[0,0]
panadol_spolu = paracetamol.sort_values(by='LIEK_popis').query('STAT_popis == "Česko"').iloc[1:12,:].groupby('STAT_popis').sum()
panadol = panadol_spolu.iloc[0,0]
display(paralen)
display(panadol)
print(paralen / panadol)

2920149.5220000003

347312.855

8.40783598983113


In [34]:
'''
from pywaffle import Waffle
fig = plt.figure(
    FigureClass=Waffle,
    rows=5,
    values=[2*paralen/panadol, 2],
    icons = 'pills',
    #block_arranging_style='new-line',
    vertical=False
)
'''

"\nfrom pywaffle import Waffle\nfig = plt.figure(\n    FigureClass=Waffle,\n    rows=5,\n    values=[2*paralen/panadol, 2],\n    icons = 'pills',\n    #block_arranging_style='new-line',\n    vertical=False\n)\n"

# Ibuprofen

In [42]:
ibuprofen = top20tab["M01AE01"].sort_values(by='Počet balení', ascending=False).head(30)

for i,popis in enumerate(ibuprofen['LIEK_popis']):
    if 'IBALGIN 400' in popis:
        ibuprofen.iloc[i,3] = 'IBALGIN 400'
    elif 'Ibalgin 400' in popis:
        ibuprofen.iloc[i,3] = 'IBALGIN 400'
    elif 'BRUFEN 400' in popis:
        ibuprofen.iloc[i,3] = 'BRUFEN 400'
    elif 'Brufen 400' in popis:
        ibuprofen.iloc[i,3] = 'BRUFEN 400'
    elif 'BRUFEN sirup' in popis:
        ibuprofen.iloc[i,3] = 'BRUFEN sirup'
    elif 'BRUFEN retard 800' in popis:
        ibuprofen.iloc[i,3] = 'BRUFEN retard 800'
    elif 'BRUFEN 600' in popis:
        ibuprofen.iloc[i,3] = 'BRUFEN 600'
    elif 'MIG-400' in popis:
        ibuprofen.iloc[i,3] = 'MIG-400'
    elif 'NUROFEN pre deti' in popis:
        ibuprofen.iloc[i,3] = 'NUROFEN pre deti'
    elif 'NUROFEN PRE DETI' in popis:
        ibuprofen.iloc[i,3] = 'NUROFEN pre deti'
    elif 'Nurofen pre deti' in popis:
        ibuprofen.iloc[i,3] = 'NUROFEN pre deti'
    elif 'NUROFEN PRE DETI' in popis:
        ibuprofen.iloc[i,3] = 'NUROFEN pre deti'
    elif 'IBALGIN RAPIDCAPS' in popis:
        ibuprofen.iloc[i,3] = 'IBALGIN RAPIDCAPS'
    elif 'IBALGIN 200' in popis:
        ibuprofen.iloc[i,3] = 'IBALGIN 200'
    elif 'IBALGIN 600' in popis:
        ibuprofen.iloc[i,3] = 'IBALGIN 600'
    elif 'IBALGIN FAST' in popis:
        ibuprofen.iloc[i,3] = 'IBALGIN FAST'
    elif 'NUROFEN Liquid Capsules' in popis:
        ibuprofen.iloc[i,3] = 'NUROFEN Liquid Capsules'
    elif 'IBUPROFEN 400' in popis:
        ibuprofen.iloc[i,3] = 'IBUPROFEN 400'
    elif 'IBALGIN SUS' in popis:
        ibuprofen.iloc[i,3] = 'IBALGIN SUS'
    elif 'MIG' in popis:
        ibuprofen.iloc[i,3] = 'MIG Junior'
    elif 'NUROFEN FORTE' in popis:
        ibuprofen.iloc[i,3] = 'NUROFEN FORTE'
    elif 'NUROFEN tbl' in popis:
        ibuprofen.iloc[i,3] = 'NUROFEN tbl'

ibuprofen = ibuprofen.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(ibuprofen)

,LIEK_popis,STAT_popis,VYROBCA_popis,Počet balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,BRUFEN 400,Francúzsko,Mylan Laboratories SAS,260398.011,1000652.93,8.524806
1,BRUFEN 600,Francúzsko,Mylan Laboratories SAS,28425.100,88020.41,3.096573
2,IBALGIN 200,Slovensko,sanofi-aventis Slovakia s.r.o.,115937.580,274563.27,2.368199
3,IBALGIN 400,Slovensko,sanofi-aventis Slovakia s.r.o.,925557.576,3503655.19,17.731865
4,IBALGIN 600,Slovensko,sanofi-aventis Slovakia s.r.o.,36402.670,64870.46,1.782025
5,IBALGIN FAST,Slovensko,sanofi-aventis Slovakia s.r.o.,32353.100,108560.61,3.355493
6,IBALGIN RAPIDCAPS,Slovensko,sanofi-aventis Slovakia s.r.o.,195327.956,899915.02,8.908333
7,IBALGIN SUS,Slovensko,sanofi-aventis Slovakia s.r.o.,16038.000,52677.24,3.284527
8,IBUPROFEN 400,Nemecko,STADA Arzneimittel AG,87787.140,228675.29,4.618850
9,MIG Junior,Nemecko,Berlin-Chemie AG,12276.300,34815.97,2.836031


In [44]:
fig = px.scatter(ibuprofen, x='LIEK_popis', y='Počet balení', size='Cena úhrada_PAC/bal', color='STAT_popis', title='Ibuprofen')
fig.update_xaxes(tickangle=45)
fig.show()

Z grafu vidime, ze najpredavanejsi liek s ibuprofenom je slovensky Ibalgin 400. Z ceskych liekov bol najpredavanejsi Nurofen pre deti. To, ze bol druhy najpredavanejsi je mozno sposobene tym ze je prave pre deti. Hodnoty cien su vyrazne skreslene, co moze byt sposobene tym, ze mozno vela pacientom lieky nepreplacala poistovna.  